In [1]:
directory = '/home/hindupur.v/varsha_hpp/dataset/colon_n'
filename = 'colonn'

In [1]:
!python --version

Python 3.9.12


In [8]:
conda create --name myenv python=3.9

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.13.0
  latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c defaults conda


^C

CondaError: KeyboardInterrupt


Note: you may need to restart the kernel to use updated packages.


In [7]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable
  Using cached torch-2.2.2-cp39-cp39-manylinux1_x86_64.whl (755.5 MB)


In [8]:
!pip install torchvision

Defaulting to user installation because normal site-packages is not writeable
  Using cached torchvision-0.17.2-cp39-cp39-manylinux1_x86_64.whl (6.9 MB)


In [9]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp

In [ ]:
def setup(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    dist.init_process_group("nccl", rank=rank, world_size=world_size)

def cleanup():
    dist.destroy_process_group()

def train_model(rank, world_size, num_epochs=5):
    setup(rank, world_size)
    
    # Set device
    device = torch.device(f'cuda:{rank}')

    # Define transforms
    train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Load the dataset
    data_dir = "../data/SpringBoard/lung_colon_image_set"
    dataset = datasets.ImageFolder(data_dir, transform=train_transforms)
    
    # Splitting the dataset could be done before running this script to ensure it's evenly distributed
    train_sampler = DistributedSampler(dataset, num_replicas=world_size, rank=rank, shuffle=True)
    train_loader = DataLoader(dataset, batch_size=128, sampler=train_sampler)
    
    # Model setup
    model = models.resnet101(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 5)
    model = model.to(device)
    model = nn.parallel.DistributedDataParallel(model, device_ids=[rank])
    
    # Criterion and optimizer
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(model.module.fc.parameters(), lr=0.001)
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
        
        # Calculate and print the average loss per epoch
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Rank {rank}, Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
    
    cleanup()

def main():
    world_size = torch.cuda.device_count()
    mp.spawn(train_model,
             args=(world_size,),
             nprocs=world_size,
             join=True)

if __name__ == "__main__":
    main()


In [5]:
conda activate myenv


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.



Note: you may need to restart the kernel to use updated packages.


In [2]:
conda update python

Solving environment: | 

Updating python is constricted by 

anaconda -> requires python==3.9.12=h12debd9_0

If you are sure you want an update of your package either try `conda update --all` or install a specific version of the package you want using `conda install <pkg>=<version>`

done


==> WARNING: A newer version of conda exists. <==
  current version: 4.13.0
  latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /shared/centos7/anaconda3/2022.05

  added / updated specs:
    - python


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-pack-0.7.1           |   py39h06a4308_0          51 KB
    ------------------------------------------------------------
                                           Total:          51 KB

The following packages will be UPDATED:

  conda-pack         pkgs/

In [6]:
pip list

Package                       Version
----------------------------- --------------------
absl-py                       2.1.0
aiohttp                       3.8.1
aiosignal                     1.2.0
alabaster                     0.7.12
anaconda-client               1.9.0
anaconda-navigator            2.1.4
anaconda-project              0.10.2
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.2
astroid                       2.6.6
astropy                       5.0.4
asttokens                     2.0.5
astunparse                    1.6.3
async-timeout                 4.0.1
atomicwrites                  1.4.0
attrs                         21.4.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.9.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
backports.tempfile            1.0
backpor

In [1]:
pip freeze | grep -vE "(pip|setuptools|wheel)" | xargs pip uninstall -y

ERROR: Invalid requirement: '@'
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip uninstall -y tensorflow

Found existing installation: tensorflow 2.15.0.post1
Uninstalling tensorflow-2.15.0.post1:
  Successfully uninstalled tensorflow-2.15.0.post1
Found existing installation: requests 2.27.1
Uninstalling requests-2.27.1:
ERROR: Exception:
Traceback (most recent call last):
  File "/shared/centos7/anaconda3/2022.05/lib/python3.9/shutil.py", line 823, in move
    os.rename(src, real_dst)
OSError: [Errno 18] Invalid cross-device link: '/shared/centos7/anaconda3/2022.05/lib/python3.9/site-packages/requests-2.27.1.dist-info/' -> '/tmp/pip-uninstall-kazix0g5'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/shared/centos7/anaconda3/2022.05/lib/python3.9/site-packages/pip/_internal/cli/base_command.py", line 173, in _main
    status = self.run(options, args)
  File "/shared/centos7/anaconda3/2022.05/lib/python3.9/site-packages/pip/_internal/commands/uninstall.py", line 93, in run
    uninstall_pathset = req.uninstall(
  File "/share